In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [0]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json("/mnt/formula1/raw/drivers.json")

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [0]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/formula1/processed/drivers")

In [0]:
display(spark.read.parquet("/mnt/formula1/processed/drivers"))